In [ ]:
%pip install -q social_agents/

In [ ]:
%pip show social_agents

In [ ]:
%pip install -U pydantic
%pip install -U "langchain[together]"
%pip install -U "langchain"
%pip install -U "langgraph"

In [ ]:
import tqdm
# Reading the data
import pandas as pd

from social_agents.utils import get_st_data


for key, line in tqdm.tqdm(get_st_data("sample").items()):
    print(key)

    print(line['intervention'])
    input_text = line['intervention']
    break


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

## Prerequisites
env, LLM, ...

# Shared Task: Critical thinking generation
## Loadind Data


In [ ]:
import os, getpass
from langchain.chat_models import init_chat_model
from IPython.display import Image, display

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("MISTRAL_API_KEY")
_set_env("TOGETHER_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "shared_task_critical_questions"

#init_chat_model("meta-llama/Llama-3.2-3B-Instruct-Turbo", model_provider="together", temperature= 0.7)


In [ ]:

#llm_name = "gpt-4o-mini-2024-07-18"# "o3-mini-2025-01-31"
#llm_name_short = "gpt-4o-mini_"
llm_dic = {
    #"openai_o3mini": "o3-mini-2025-01-31",
    #"gpt-4o-mini_": "gpt-4o-mini-2024-07-18",
    "llama8b_": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "mistral24b_": "mistral-small-2503",#"mistralai/Mixtral-8x7B-Instruct-v0.1",
    "llama70b_": "meta-llama/Llama-3.3-70B-Instruct-Turbo"

}

##  1. Zero Shot LLM

In [ ]:
llm_dic

In [ ]:

from social_agents.agent_builder import BasicCQModel


for llm_key, llm_full in llm_dic.items():
    print(f"Running {llm_key}")
    basic_agent = BasicCQModel(
        model_thread_id=f"{llm_key}_zero-shot_temp0",
        llm_name = llm_full,
        llm_num = 1,
        experiment_name= f"{llm_key}_zero-shot_temp0",
        temperature=0.0)
        
        
    #display(Image(basic_agent.graph.get_graph(xray=1).draw_mermaid_png()))

# RUN
    basic_agent.run_experiment()



## Social Agents


In [ ]:
%pip show langchain-core

### Fetching all experiment setting

In [ ]:
import pandas as pd
import os

from social_agents import helper
from social_agents.agent_builder import  SocialAgentBuilder

EXPERIMENT_SETTINGS_FILE_PATH = "output/elbaff_experiment/experiment_settings.csv"
override = False
if not os.path.exists(EXPERIMENT_SETTINGS_FILE_PATH) or override:
    print("generating exp settings file")
    exps_df = helper.generate_experiment_settings()
    exps_df.to_csv(EXPERIMENT_SETTINGS_FILE_PATH, index=False)

experiment_settings = pd.read_csv(EXPERIMENT_SETTINGS_FILE_PATH)
experiment_settings = experiment_settings.sort_values(by=["rounds", "number_of_agents"])

In [ ]:
import ast
import time
from social_agents import data_model
import ast
from tqdm import tqdm



temperature = 0.7
llm_name_short = "mistral24b_"
llm_name = llm_dic[llm_name_short]
def run_all_exp_settings(experiment_settings):
    all_done = False
    for _, row in tqdm(experiment_settings.iterrows(), total=len(experiment_settings)):
        exp_name = row["experiment_name"].format(llm_name= llm_name_short)
        if os.path.exists(f"{SocialAgentBuilder.ROOT_FOLDER}output_{exp_name}.json"):
            print("Experiment ", exp_name, "already done!")
            continue

        print("EXPERIMENT NAME: ", exp_name)

        social_agent = SocialAgentBuilder(
            model_thread_id=row["thread_id"],
            llm_name = llm_name,
            llm_num = row["number_of_agents"],
            experiment_name= exp_name,
            temperature=temperature,
            collaborative_strategy=list(ast.literal_eval(row["strategies"])),
            agent_trait_lst=list(ast.literal_eval(row["traits"])))
        #display(Image(social_agent.graph.get_graph(xray=1).draw_mermaid_png()))

        social_agent.run_experiment(data_type = "validation", save= True)
        print(f'finished {exp_name}')
    all_done = True
    return all_done
delay_in_sec = 0.1
all_done = False
while True and not all_done: 
    try:###
        all_done = run_all_exp_settings(experiment_settings=experiment_settings)
    except Exception as e:
        all_done = False
        print(f"Exception caught: {e}, retrying in {delay_in_sec} seconds...")
        time.sleep(delay_in_sec)


In [ ]:

import glob
import json
len(glob.glob("output/elbaff_experiment/final_states/llama8b*_4R*arg*.json")
)

In [2]:
import glob

l=glob.glob("output/elbaff_experiment/final_states/2S*arg*.json")
l.sort()
len(l)
print(round((len(l)*100)/(186*3)), "%")

9 %


In [3]:
def print_progress_bar(percent: float, width: int = 50):
    """
    Print a progress bar to the notebook output.

    Args:
        percent (float): Completion percentage (0.0 to 1.0).
        width (int): Width of the progress bar in characters.
    """
    filled = int(width * percent)
    bar = "█" * filled + "-" * (width - filled)
    print(f"[{bar}] {percent * 100:.1f}%", end="\r")


In [ ]:
import time
from datetime import datetime
i,port glob
# Number of iterations (set to None or a high number if you want it to run indefinitely)
num_runs = 6*3  # e.g., for 1 hour (6 x 10 minutes)

for i in range(num_runs):
    # Your code here
    l=glob.glob("output/elbaff_experiment/final_states/2S*arg*.json")
    #print(round((len(l)*100)/(186*3)), "%")
    print_progress_bar((len(l)/(186*3)))
    # Wait for 10 minutes (600 seconds)
    if i < num_runs - 1:  # Don't sleep after the last run
        time.sleep(6)


NameError: name 'glob' is not defined